In [1]:
from policyengine.countries import US
from policyengine.utils.charts import plotly_json_to_fig
from blank_slate_ubi_us.common import blank_slate_ubi, prepare_simulation
from ubicenter import format_fig

us = US()

blank_slate = blank_slate_ubi()
blank_slate_baseline = prepare_simulation()
blank_slate_reform = dict(reform=blank_slate, baseline_reform=blank_slate_baseline)

baseline, reformed = us._get_microsimulations(blank_slate)

charts = us.population_reform(blank_slate)

## Figure 1

In [2]:
format_fig(plotly_json_to_fig(charts["intra_income_decile_chart"]).update_layout(
    title="Distribution of gains and losses by income decile under Blank Slate UBI",
))

## Figure 2

In [3]:
from blank_slate_ubi_us.charts.age_winners import age_winner_chart

format_fig(plotly_json_to_fig(age_winner_chart(baseline, reformed, us.results_config)).update_layout(
    title="Distribution of gains and losses by age under Blank Slate UBI",
))

## Figure 3

In [4]:
from blank_slate_ubi_us.charts.state_choropleth import us_state_choropleth

us_state_choropleth(baseline, reformed)

## Figure 4

In [5]:
format_fig(plotly_json_to_fig(charts["poverty_chart"]).update_layout(
    title="Change to poverty rates by age group under Blank Slate UBI",
))

In [6]:
import numpy as np
poverty_rate = lambda sim: sim.calc("spm_unit_is_in_spm_poverty", map_to="person").mean()
deep_poverty_rate = lambda sim: sim.calc("spm_unit_is_in_deep_spm_poverty", map_to="person").mean()

def poverty_gap_fn(sim, deep):
    distance = sim.calc("spm_unit_spm_threshold") * (1 if not deep else 0.5) - sim.calc("spm_unit_net_income")
    positive_distance = np.maximum(0, distance)
    weight = sim.calc("spm_unit_weight").values
    return (weight * positive_distance).sum()

poverty_gap = lambda sim: poverty_gap_fn(sim, False)
deep_poverty_gap = lambda sim: poverty_gap_fn(sim, True)
metric_names = ["Poverty rate", "Deep poverty rate", "Poverty gap", "Deep poverty gap"]

import pandas as pd

# For each metric, add columns "Name", "Baseline", "Reformed", "Absolute change", "Relative change".

df = pd.DataFrame({
    "Name": metric_names,
    "Baseline": [poverty_rate(baseline) / 1e-2, deep_poverty_rate(baseline) / 1e-2, poverty_gap(baseline) / 1e9, deep_poverty_gap(baseline) / 1e9],
    "Reformed": [poverty_rate(reformed) / 1e-2, deep_poverty_rate(reformed) / 1e-2, poverty_gap(reformed) / 1e9, deep_poverty_gap(reformed) / 1e9],
})

df["Change"] = df.Reformed - df.Baseline
df["Relative change"] = (df.Reformed / df.Baseline - 1) * 100

df.Baseline = df.Baseline.round(1)
df.Reformed = df.Reformed.round(1)
df.Change = df.Change.round(1)
df["Relative change"] = df["Relative change"].round(1)

df

,Name,Baseline,Reformed,Change,Relative change
0,Poverty rate,8.9,3.9,-4.9,-55.9
1,Deep poverty rate,2.8,0.4,-2.5,-87.1
2,Poverty gap,105.8,38.3,-67.5,-63.8
3,Deep poverty gap,16.9,2.8,-14.1,-83.6


In [7]:
format_fig(plotly_json_to_fig(charts["rel_income_decile_chart"]).update_layout(
    title="Change to net income by income decile under Blank Slate UBI",
))

In [8]:
format_fig(plotly_json_to_fig(charts["inequality_chart"]).update_layout(
    title="Changes to inequality metrics under Blank Slate UBI",
))

In [9]:
from blank_slate_ubi_us.charts.state_poverty_choropleth import us_state_poverty_choropleth

us_state_poverty_choropleth(baseline, reformed)

In [10]:
from blank_slate_ubi_us.charts.program_winners import program_winner_chart

program_winner_chart(baseline, reformed, us.results_config)

In [11]:
household = {
    "people": {
        "adult": {
            "age": {2022: 25},
        },
        "adult_2": {
            "age": {2022: 25},
        },
        "child": {
            "age": {2022: 5},
        }
    },
    "tax_units": {
        "tax_unit": {
            "members": ["adult", "adult_2", "child"],
        }
    },
    "marital_units": {
        "marital_unit": {
            "members": ["adult", "adult_2"],
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["adult", "adult_2", "child"],
        }
    },
    "households": {
        "household": {
            "members": ["adult", "adult_2", "child"],
            "state_code": {2022: "MA"},
            "zip_code": {2022: "01001"},
        }
    },
}

fig = us.household_variation(dict(reform=blank_slate, baseline_reform=blank_slate_baseline, household=household))

In [12]:
format_fig(plotly_json_to_fig(fig["budget_chart"]).update_layout(
    title="Net income by employment income under Blank Slate UBI",
))

In [13]:
format_fig(plotly_json_to_fig(fig["mtr_chart"]).update_layout(
    title="Marginal tax rates under Blank Slate UBI",
    yaxis_range=(-0.5, 1),
))